# Data transformation

# 1. Introduction and purpose

This notebook has the purpose of transformating/cleaning the data obtained in the extraction step from `notebooks/1.data_extraction.ipynb`.

Transformation goals are:
- Cleaning and formatting of tables extracted
- Generation of Technical Indicators from OHLCV data

Once data is obtained perform, preliminary analysis, cleaning and EDA.

- IPO Data (dont think it is used) through request and pd.read_html. Stack all years IPO. Make return analysis.
- Get OHLCV data. (In his case 33 tickers, seen later) Get year, month, date from index and desired growth lags (pct_change 1d, 5d, 30d). Create predictors from lag and binary lag.
- Get macroindicators. Get desired lags.
- Get macroindicator Volatility
- Get GOLD. You can get futures GC=F or spot XAU/USD
- Get WTI Crude Oil futures (CL=F) and Brent Oil futures (BZ=F)
- Get BTC-USD
- Get eurostat data. He looks just at the curve from 2024-04-18. I do not know who one would use this. Maybe it would be better to use the parameters of each curve.
- Fundamental indicators. (Just join the data properly and use it for first model. Do analysis later) 4 years history. Looking at attributes (for example SEC filings) can be helpful for me to know how to structure my scraped information. How to join these dates? He keeps net_income_margin, ebitda_margin and EPS. Same, how to join previous data about fundamentals? 
- In the videos he mentions the ratios used to predict enterprise "fair value": EPS, Revenue, Net Income, Profit Margin, EBITDA, P/E, etc. Better look at that slide. This is more important for long-term.
- Download all tickers to be used. Uses tickers from US, EU and India. He calculates the mean growth for the 365d period with respect to previous year, each year - compare with mean growth for that year.
- TA-lib indicators. Will have to look how they are built to properly use them. Convert inputs to floats for TA-lib.
- Merge


# 2. Data transformation

In [ ]:
%load_ext autoreload
%autoreload 2

# Data preparation
import numpy as np
import polars as pl
import pandas as pd

# Data
import sys
sys.path.append("..")

from src.support.data_transformation import TickerExtender, TechnicalIndicators, FileHandler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. OHLCV

## 1.1 Adding simple temporal growth and simple technical indicators

In [42]:
ticker_extender = TickerExtender()
file_handler = FileHandler()

stocks_df_list = ticker_extender.transform_daily_tickers_parallel("../data/extracted/OHLCV")

## 1.2 Adding Technical Indicators

In [ ]:
# Calculator of 1. momentum, 2. volume/volatility/cycle and 3. patterns technical indicators
tech_indicators_calculator = TechnicalIndicators()

# supress warnings
pd.options.mode.chained_assignment = None  # default='warn'

# adding Momentum + Pattern + Volume features to all tickers - one by one
merged_df_with_tech_ind = pd.DataFrame()

for i, stocks_df in enumerate(stocks_df_list):

  ticker_name = stocks_df["symbol"].unique()[0]

  print(f'{i}/{len(stocks_df_list)} Current ticker is {ticker_name}')

  current_ticker_data = stocks_df[stocks_df["symbol"].isin([ticker_name])]

  # need to have same 'utc' time on both sides
  # https://stackoverflow.com/questions/73964894/you-are-trying-to-merge-on-datetime64ns-utc-and-datetime64ns-columns-if-yo
  current_ticker_data['date']= pd.to_datetime(current_ticker_data['date'], utc=True)

  # Momentum technical indicators
  df_current_ticker_momentum_indicators = tech_indicators_calculator.talib_get_momentum_indicators_for_one_ticker(current_ticker_data)
  df_current_ticker_momentum_indicators["date"]= pd.to_datetime(df_current_ticker_momentum_indicators['date'], utc=True)

  # Vol
  df_current_ticker_volume_indicators = tech_indicators_calculator.talib_get_volume_volatility_cycle_price_indicators(current_ticker_data)
  df_current_ticker_volume_indicators["date"]= pd.to_datetime(df_current_ticker_volume_indicators['date'], utc=True)

  df_current_ticker_pattern_indicators = tech_indicators_calculator.talib_get_pattern_recognition_indicators(current_ticker_data)
  df_current_ticker_pattern_indicators["date"]= pd.to_datetime(df_current_ticker_pattern_indicators['date'], utc=True)

  # merge to one df
  m1 = pd.merge(current_ticker_data, df_current_ticker_momentum_indicators.reset_index(), how = 'left', on = ["date","symbol"], validate = "one_to_one")
  m2 = pd.merge(m1, df_current_ticker_volume_indicators.reset_index(), how = 'left', on = ["date","symbol"], validate = "one_to_one")
  m3 = pd.merge(m2, df_current_ticker_pattern_indicators.reset_index(), how = 'left', on = ["date","symbol"], validate = "one_to_one")

  merged_df_with_tech_ind = pd.concat([merged_df_with_tech_ind,m3], ignore_index = False)

merged_df_with_tech_ind

0/33 Current ticker is INFY
1/33 Current ticker is IBN
2/33 Current ticker is SBIN.NS
3/33 Current ticker is LT.NS
4/33 Current ticker is LICI.NS
5/33 Current ticker is BHARTIARTL.NS
6/33 Current ticker is HINDUNILVR.NS
7/33 Current ticker is RELIANCE.NS
8/33 Current ticker is TCS.NS
9/33 Current ticker is ITC.NS
10/33 Current ticker is HDB
11/33 Current ticker is MSFT
12/33 Current ticker is GOOG
13/33 Current ticker is V
14/33 Current ticker is JPM
15/33 Current ticker is BRK-B
16/33 Current ticker is AMZN
17/33 Current ticker is LLY
18/33 Current ticker is AAPL
19/33 Current ticker is META
20/33 Current ticker is AVGO
21/33 Current ticker is NVDA
22/33 Current ticker is NVO
23/33 Current ticker is SAP
24/33 Current ticker is CDI.PA
25/33 Current ticker is TTE
26/33 Current ticker is IDEXY
27/33 Current ticker is OR.PA
28/33 Current ticker is ASML
29/33 Current ticker is RMS.PA
30/33 Current ticker is MC.PA
31/33 Current ticker is SIE.DE
32/33 Current ticker is ACN


,close,high,low,open,volume,symbol,year,month,weekday,date,...,cdlspinningtop,cdlstalledpattern,cdlsticksandwich,cdltakuru,cdltasukigap,cdlthrusting,cdltristar,cdlunique3river,cdlupsidegap2crows,cdlxsidegap3methods
0,0.468856,0.500112,0.373834,0.373834,172512000.0,INFY,1999,3,3,1999-03-11 00:00:00+00:00,...,0,0,0,0,0,0,0,0,0,0
1,0.466355,0.487610,0.430097,0.470106,39897600.0,INFY,1999,3,4,1999-03-12 00:00:00+00:00,...,0,0,0,0,0,0,0,0,0,0
2,0.423845,0.467605,0.421345,0.467605,12672000.0,INFY,1999,3,0,1999-03-15 00:00:00+00:00,...,0,0,0,0,0,0,0,0,0,0
3,0.420094,0.442599,0.420094,0.427596,9984000.0,INFY,1999,3,1,1999-03-16 00:00:00+00:00,...,0,0,0,0,0,0,0,0,0,0
4,0.422595,0.447601,0.417594,0.420094,15449600.0,INFY,1999,3,2,1999-03-17 00:00:00+00:00,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5905,349.790009,358.000000,348.739990,355.029999,2830000.0,ACN,2025,1,4,2025-01-10 00:00:00+00:00,...,0,0,0,0,0,0,0,0,0,0
5906,349.140015,350.910004,344.369995,345.130005,2711200.0,ACN,2025,1,0,2025-01-13 00:00:00+00:00,...,0,0,0,0,0,0,0,0,0,0
5907,348.989990,352.519989,345.630005,351.209991,1825400.0,ACN,2025,1,1,2025-01-14 00:00:00+00:00,...,0,0,0,0,0,0,0,0,0,0
5908,349.730011,355.200012,349.059998,352.350006,2615900.0,ACN,2025,1,2,2025-01-15 00:00:00+00:00,...,0,0,0,0,0,0,0,0,0,0


# 2. Macroindicators

## 2.1 Indices

### 2.1.1 DAX (Germany)


### 2.1.2 S&P500 (US)


### 2.1.3 Dow Jones (US)


### 2.1.4 EPI (India)
